In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import os


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
eat_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/강남구_식당_detail.csv")
eat_review = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/강남구_식당_review.csv")

In [6]:
eat_info

,Unnamed: 0,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점,lat,lng
0,1,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,만원 미만,25,4.0,37.520699,127.019975
1,2,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,만원 미만,16,3.8,37.519766,127.020477
2,3,딸부자네불백,서울특별시 강남구 강남대로158길 21,고기 요리,만원 미만,15,3.8,37.519231,127.020512
3,4,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,만원 미만,38,3.6,37.519027,127.023549
4,5,목포집,서울특별시 강남구 강남대로152길 13,닭 / 오리 요리,만원 미만,11,3.6,37.517342,127.020489
...,...,...,...,...,...,...,...,...,...
810,811,요술포차,서울특별시 강남구 테헤란로19길 5,일반 주점,2만원-3만원,5,NaN,37.500368,127.033440
811,812,전통주갤러리,서울특별시 강남구 테헤란로5길 51-20,전통 주점 / 포차,2만원-3만원,2,NaN,37.501505,127.027861
812,813,더블린테라스,서울특별시 강남구 강남대로94길 27-15 태리빌딩,치킨 / 호프 / 펍,2만원-3만원,1,NaN,37.500657,127.029090
813,814,술술술,서울특별시 강남구 강남대로98길 12-5,일반 주점,2만원-3만원,2,NaN,37.500376,127.027961


In [7]:
eat_review

,Unnamed: 0,가게이름,리뷰아이디,리뷰,맛평가
0,1,모터시티,Mike Kim,Love motor city pizza,맛있다
1,2,김북순큰남비집,철준,맛있음,맛있다
2,3,김북순큰남비집,돼후니,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...",맛있다
3,4,김북순큰남비집,호리,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,맛있다
4,5,김북순큰남비집,하몽이,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,맛있다
...,...,...,...,...,...
17182,17183,깐부치킨,Crisp.,깐부 처음가서 그런데 구운마늘치킨 먹음.. 맛은 있대 좀 많이 기름진거 같았어요.,괜찮다
17183,17184,깐부치킨,Miriam Park,We ordered the garlic soy chicken. The chicken...,괜찮다
17184,17185,모터시티,호두꽈자,포치즈홀스맨 24900그릴드 치킨 아보카도 샐러드 16900 (발사믹)[공간]생긴 ...,괜찮다
17185,17186,모터시티,이영훈,디트로이트 스타일 피자 처음먹어보는데 맛있어요 ㅋㅋ 가격은 좀 나가지만 토마토 소스...,맛있다


In [8]:
eat_info.columns

Index(['Unnamed: 0', '가게이름', '가게 주소', '음식 종류', '가격 범위', '리뷰개수', '평점', 'lat',
       'lng'],
      dtype='object')

In [9]:
eat_info.rename(columns={'가게이름': 'name'}, inplace =True)
eat_info.rename(columns={'리뷰': 'review'}, inplace =True)
eat_info.rename(columns={'평점': 'rating'}, inplace =True)
eat_info.rename(columns={'리뷰개수': 'rvw_cnt'}, inplace =True)
eat_info.rename(columns={'가게 주소': 'address'}, inplace =True)
eat_info.rename(columns={'음식 종류': 'category'}, inplace =True)
eat_info.rename(columns={'가격 범위': 'price'}, inplace =True)




eat_review.rename(columns={'가게이름': 'res_name'}, inplace =True)
eat_review.rename(columns={'리뷰아이디':  'user_name'}, inplace =True)
eat_review.rename(columns={'리뷰':  'review'}, inplace =True)
eat_review.rename(columns={'맛평가':  'rating'}, inplace =True)

eat_info.drop(['Unnamed: 0'],axis=1, inplace=True)
eat_review.drop(['Unnamed: 0'],axis=1, inplace=True)



In [10]:
eat_info.columns

Index(['name', 'address', 'category', 'price', 'rvw_cnt', 'rating', 'lat',
       'lng'],
      dtype='object')

In [11]:
eat_info = eat_info[eat_info['name'].notnull()]
eat_review = eat_review[eat_review['user_name'].notnull()]
eat_info.reset_index(drop=True,inplace=True)
eat_review.reset_index(drop=True,inplace=True)

In [12]:
#delete overlapped data
for id in eat_info.index:
    if list(eat_info['name']).count(eat_info.loc[id,'name'])>1:
        eat_info.drop(id,inplace=True)

In [13]:
#label encoding, Place_name -> p_id, User_name -> u_id
le = LabelEncoder()
le.fit(eat_info['name'])
eat_info['p_id'] = le.transform(eat_info['name'])
eat_review['p_id'] = le.transform(eat_review['res_name'])

user_le = LabelEncoder()
user_le.fit(eat_review['user_name'])
eat_review['u_id'] = user_le.transform(eat_review['user_name'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [14]:
# delete punctuation marks
def delete_punctuation_marks_review(review):
    review = review.replace('.', '. ');
    review = review.replace('!', '! ');
    review = review.replace('(', '( ');
    review = review.replace(')', ') ');
    review = review.replace('^', '');
    review = review.replace('*', '');
    review = review.replace('-', ' ');
    review = review.replace('\n', '. ');
    review = review.replace('\"', '. ');
    return review

In [15]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id
0,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,만원 미만,25,4.0,37.520699,127.019975,88
1,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,만원 미만,16,3.8,37.519766,127.020477,166
3,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,만원 미만,38,3.6,37.519027,127.023549,322
5,스쿨푸드,서울특별시 강남구 도산대로 139,기타 한식,만원 미만,11,3.5,37.518213,127.023711,398
6,송옥,서울특별시 강남구 강남대로154길 43 2F,국수 / 면 요리,만원 미만,11,3.1,37.518625,127.022385,376


In [16]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,모터시티,Mike Kim,Love motor city pizza,맛있다,243,559
1,김북순큰남비집,철준,맛있음,맛있다,88,3735
2,김북순큰남비집,돼후니,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...",맛있다,88,1961
3,김북순큰남비집,호리,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,맛있다,88,4100
4,김북순큰남비집,하몽이,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,맛있다,88,3990


In [17]:
eat_info['reviews'] = ''


In [18]:
# from requests import Request
# from requests import Session


# def KOtoEN(review):
#     s=Session()

#     headers ={
#         "X-Naver-Client-Id": "NsHid2ZsKcP5QclyL66L",
#         "X-Naver-Client-Secret": "OPb4L85YOo",
#     }

#     payload ={
#         'source':'ko',
#         'target':'en',
#         'text': review,
#     }

#     url ="https://openapi.naver.com/v1/papago/n2mt"
#     # "https://openapi.naver.com/v1/language/translate"

#     req =Request('POST', url, data=payload, headers=headers)
#     prepped =req.prepare()
#     res =s.send(prepped)
#     print(res.json())
#     # result =res.json()['message']['result']['translatedText']
#     return result

In [19]:
review_1=eat_review['review'][1]
review_1

'맛있음'

In [20]:
# KOtoEN(review_1)

In [21]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = KOtoEN(review_pre)
#     except:
#         review_ts = ''

#     print(review_ts)
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [22]:
# ts = Translator()

# ts.translate(review_1, source='ko', target='en', verbose=False)

In [23]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = ts.translate(review_pre, source='ko', target='en', verbose=False)
#     except:
#         review_ts = ''
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [24]:
for id in eat_info['p_id']:
    review_all = ''
    
    for review in eat_review[eat_review['p_id']==id]['review']:
        review_all+=review +'. '
        
    eat_info.loc[eat_info['p_id']==id,'reviews'] = review_all

In [25]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,만원 미만,25,4.0,37.520699,127.019975,88,"맛있음. #돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : ..."
1,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,만원 미만,16,3.8,37.519766,127.020477,166,점점 망해가는 느낌잠점 책상도 끈적끈적해지는것 같고...맛은 그럭저럭인편. 가격에 ...
3,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,만원 미만,38,3.6,37.519027,127.023549,322,떡볶이가 너무 좋아서 사먹어봤어요. 즉석 떡볶이가 참 맛있어요ㅎ약간 매운 맛인데도 ...
5,스쿨푸드,서울특별시 강남구 도산대로 139,기타 한식,만원 미만,11,3.5,37.518213,127.023711,398,오랜만에 스쿨푸드에서 배달시켜 먹었어요. 15년 전 즈음엔 가로수길에 와서 스쿨푸드...
6,송옥,서울특별시 강남구 강남대로154길 43 2F,국수 / 면 요리,만원 미만,11,3.1,37.518625,127.022385,376,판메밀(2판) / 돈가스. 한국식 판메밀을 판매하는 곳으로 역사도 있는 집으로 보인...


In [26]:
eat_info['price'].unique()

array(['만원 미만', '만원-2만원', '2만원-3만원', '3만원-4만원', '4만원 이상'], dtype=object)

In [27]:
eat_info['price'].replace('만원 미만',5000, inplace=True)
eat_info['price'].replace('만원-2만원',15000, inplace=True)
eat_info['price'].replace('2만원-3만원',25000, inplace=True)
eat_info['price'].replace('3만원-4만원',35000, inplace=True)
eat_info['price'].replace('4만원 이상',50000, inplace=True)

In [28]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,모터시티,Mike Kim,Love motor city pizza,맛있다,243,559
1,김북순큰남비집,철준,맛있음,맛있다,88,3735
2,김북순큰남비집,돼후니,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...",맛있다,88,1961
3,김북순큰남비집,호리,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,맛있다,88,4100
4,김북순큰남비집,하몽이,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,맛있다,88,3990


In [29]:
eat_review['rating'].unique()

array(['맛있다', '별로', '괜찮다'], dtype=object)

In [30]:
eat_review['rating'].replace('맛있다',5, inplace=True)
eat_review['rating'].replace('괜찮다',3, inplace=True)
eat_review['rating'].replace('별로',0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [31]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,5000,25,4.0,37.520699,127.019975,88,"맛있음. #돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : ..."
1,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,5000,16,3.8,37.519766,127.020477,166,점점 망해가는 느낌잠점 책상도 끈적끈적해지는것 같고...맛은 그럭저럭인편. 가격에 ...
3,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,5000,38,3.6,37.519027,127.023549,322,떡볶이가 너무 좋아서 사먹어봤어요. 즉석 떡볶이가 참 맛있어요ㅎ약간 매운 맛인데도 ...
5,스쿨푸드,서울특별시 강남구 도산대로 139,기타 한식,5000,11,3.5,37.518213,127.023711,398,오랜만에 스쿨푸드에서 배달시켜 먹었어요. 15년 전 즈음엔 가로수길에 와서 스쿨푸드...
6,송옥,서울특별시 강남구 강남대로154길 43 2F,국수 / 면 요리,5000,11,3.1,37.518625,127.022385,376,판메밀(2판) / 돈가스. 한국식 판메밀을 판매하는 곳으로 역사도 있는 집으로 보인...
...,...,...,...,...,...,...,...,...,...,...
810,요술포차,서울특별시 강남구 테헤란로19길 5,일반 주점,25000,5,NaN,37.500368,127.033440,528,예약하러고 전화 3번 했을 때 알아봤어야 했는데... 저녁에 3테이블정도밖에 없을 ...
811,전통주갤러리,서울특별시 강남구 테헤란로5길 51-20,전통 주점 / 포차,25000,2,NaN,37.501505,127.027861,591,다양한 전통주를 시음할 수 있는 공간전통주 무료시음회도 하고 판매도 하는 곳입니다....
812,더블린테라스,서울특별시 강남구 강남대로94길 27-15 태리빌딩,치킨 / 호프 / 펍,25000,1,NaN,37.500657,127.029090,148,강남역에 위치한 맥주펍. CGV 뒷골목 더블린 말고 주택가 쪽에 올라가면 위치해있는...
813,술술술,서울특별시 강남구 강남대로98길 12-5,일반 주점,25000,2,NaN,37.500376,127.027961,381,오랫동안 강남역 바닥에서 술을 마셨지만 못들어본곳이었던 이곳. 왠만한 안주는 다 좋...


In [32]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,모터시티,Mike Kim,Love motor city pizza,5,243,559
1,김북순큰남비집,철준,맛있음,5,88,3735
2,김북순큰남비집,돼후니,"#돼훈맛집 #솔직돼훈맛있고, 든든하게 한식을 먹을 수 있는 곳! -맛 : 3.7점 ...",5,88,1961
3,김북순큰남비집,호리,[신사/김북순 큰남비집] 5/5오 좋다! 소리가 절로 나온 찌개 맛집이오~개인적으로...,5,88,4100
4,김북순큰남비집,하몽이,누구나 아는 맛 그러나 모두가 즐기는 맛 근처에서 뜨끈한 국물에 흰 쌀밥 생각나면 ...,5,88,3990
...,...,...,...,...,...,...
17130,깐부치킨,Crisp.,깐부 처음가서 그런데 구운마늘치킨 먹음.. 맛은 있대 좀 많이 기름진거 같았어요.,3,95,160
17131,깐부치킨,Miriam Park,We ordered the garlic soy chicken. The chicken...,3,95,570
17132,모터시티,호두꽈자,포치즈홀스맨 24900그릴드 치킨 아보카도 샐러드 16900 (발사믹)[공간]생긴 ...,3,243,4095
17133,모터시티,이영훈,디트로이트 스타일 피자 처음먹어보는데 맛있어요 ㅋㅋ 가격은 좀 나가지만 토마토 소스...,5,243,3341


In [33]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restaurant_info.csv', index=False)
eat_review.to_csv('/content/drive/MyDrive/Colab Notebooks/data/강남구_restaurant_review.csv', index=False)